# AM Laser Pathing

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
from materialite.models.grain_coarsening_model import GrainCoarseningModel
from materialite.models.convolution_model import LaserBeam
from materialite import Material
import numpy as np

The convolution, convolution_gpu, and convolution_microstructure models contain a laser beam class designed to auto-generate beam pathing. In the plot, the blue dashed line represents the scan path, and the red dots represent the discrete points along the path that would be used in an incremental model.

In [ ]:
material = Material(dimensions=[100, 100, 102], spacing=[5e-6, 5e-6, 5e-6])
laser = LaserBeam(
    material,
    laser_power=100,
    hatch_spacing=20,
    beam_x_radius=50.0e-6,
    beam_y_radius=50.0e-6,
    beam_z_radius=30.0e-6,
    num_layers=10,
    scan_offset_start=0,
    hatch_offset_start=0,
    hatch_offset_end=0,
    z_start=30,
    layer_thickness=8,
    time_between_scans=0.000001,
    time_between_layers=0.0,
    rotation_angle_increment=np.deg2rad(67),
)

laser.create_build_path(material, adjust_domain=False, domain_epsilon=5e-6)
laser.plot_beam_path()

It is able to generate relatively complex patterns with multiple layers. However, the design of the class causes points to be generated outside of the processing domain (0-100 in x and 0-100 in y in this case). This can be adjusted by adding 'adjust_domain=True'. The domain_epsilon term gives some flexibilty on how close to or far out of the domain to allow points to generate. Three examples are shown below. 

In [ ]:
laser.create_build_path(material, adjust_domain=True, domain_epsilon=5e-6)
laser.plot_beam_path()
laser.create_build_path(material, adjust_domain=True, domain_epsilon=-10e-6)
laser.plot_beam_path()
laser.create_build_path(material, adjust_domain=True, domain_epsilon=20e-6)
laser.plot_beam_path()

Geometric inputs (hatch spacing, layer thickness, offset values, and z_start) other than beam radius are in units of voxels to make it easier to sync with the material dimensions. The offset start and end values can be used to adjust where a line starts and ends for path design. Here is a case with only one layer. `create_build_path` needs to be re-called to regenerate the path. 

In [ ]:
material = Material(dimensions=[100, 50, 30], spacing=[5e-6, 5e-6, 5e-6])
laser = LaserBeam(
    material,
    laser_power=100,
    hatch_spacing=20,
    beam_x_radius=50.0e-6,
    beam_y_radius=50.0e-6,
    beam_z_radius=30.0e-6,
    num_layers=1,
    scan_offset_start=0,
    hatch_offset_start=0,
    hatch_offset_end=0,
    z_start=30,
    layer_thickness=8,
    time_between_scans=0.000001,
    time_between_layers=0.0,
    rotation_angle_increment=np.deg2rad(67),
)

laser.create_build_path(material, adjust_domain=False, domain_epsilon=5e-6)
laser.plot_beam_path()

laser.hatch_offset_end = -10
laser.hatch_offset_start = 5
laser.scan_offset_start = -5
laser.create_build_path(material, adjust_domain=False, domain_epsilon=5e-6)
laser.plot_beam_path()

The spacing between points is dependent on the laser scanning velocity. It is limited to there is no more than one voxel traveled between time steps. This is to prevent solidification issues in the convolution microstructure models. The position, time steps, and other parameters can be easily accessed. The first step has zero time and power to help initiate the simulation.

In [ ]:
print("X-position (m):", laser.x_pos[0:5])
print("Laser Power (W):", laser.beam_power[0:5])
print("Time (s):", laser.time[0:5])